# Model convertion for mobile

- Converting a Keras model using TensorFlow Lite: https://www.tensorflow.org/lite/convert/python_api#converting_a_keras_model

### Loading the model

In [1]:
import tensorflow.keras.backend as K

def f1_score(y_true, y_pred):
    """F1 score metric.
    
    The F1 score is the harmonic mean of the precision and
    recall, where an F1 score reaches its best value at 1
    (perfect precision and recall).
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    p = true_positives / (predicted_positives + K.epsilon())
    r = true_positives / (possible_positives + K.epsilon())
    return 2 * p * r / (p + r + K.epsilon())

def f1_loss(y_true, y_pred):
    """
    The best loss function for F1-score would be, of course, the metric
    itself: minimizing 1 − F1 is same as maximizing F1.
    """
    true_positives = K.sum(y_true * y_pred)
    predicted_positives = K.sum(y_pred)
    possible_positives = K.sum(y_true)
    
    p = true_positives / (predicted_positives + K.epsilon())
    r = true_positives / (possible_positives + K.epsilon())
    return 1 - (2 * p * r / (p + r + K.epsilon()))

In [2]:
from tensorflow.keras.models import load_model

model = load_model(
    'models/unet_v2.f1lo-b14-e60-lr0.001.44.hdf5',
    custom_objects={'f1_score':f1_score, 'f1_loss': f1_loss}
)

### Converting the model

In [0]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('models/unet_v2.f1lo-b14-e60-lr0.001.44.tflite', 'wb') as f:
    f.write(tflite_model)

### Converting the model with optimizations

In [4]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('models/unet_v2.f1lo-b14-e60-lr0.001.44.optimized.tflite', 'wb') as f:
    f.write(tflite_model)